Everyone goes to the mall, but not everyone spends at the mall. Some are just window shopping while others go on a mass shopping expedition.

Information on customers' gender, age, annual income and spending score was gathered to identify clusters within them to better serve them. The mall may also use this data to develop a marketing strategy!

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style("ticks")

Let's take a look at the raw data:

In [ ]:
df=pd.read_csv("../input/customer-segmentation-tutorial-in-python/Mall_Customers.csv")
df.head()

Drop the "CustomerID" column as it does not provide any insight to the clusters:

In [ ]:
df.drop(["CustomerID"],axis=1,inplace=True)

Change the "Gender" column to binary:

In [ ]:
df["Gender"]=pd.get_dummies(df["Gender"],drop_first=True)
#where 0=female and 1=male

Now let's check the data:

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum()
#no missing data, which is good

OK this is what the working dataset looks like:

In [ ]:
df.head()

In [ ]:
fig=plt.figure(figsize=(12,8))
sns.distplot(a=df["Spending Score (1-100)"],bins=40,color="mediumpurple")
plt.title("Customer Spending Distribution",size=16)

* Majority of the customers have a spending score of between 40 and 60, then again between 0 and 20 & 70 and 100.

In [ ]:
fig=plt.figure(figsize=(12,8))

ax1=plt.subplot2grid((2,3),(0,0),colspan=3)
ax2=plt.subplot2grid((2,3),(1,0))
ax3=plt.subplot2grid((2,3),(1,1))
ax4=plt.subplot2grid((2,3),(1,2))

sns.boxplot(x=df["Spending Score (1-100)"],y=df["Gender"],orient="h",palette={0:"pink",1:"skyblue"},ax=ax1)

ax2.pie(df["Gender"].value_counts(),colors=["pink","skyblue"],labels=["Females","Males"])

sns.distplot(a=df["Age"].loc[(df["Gender"]==0)],bins=20,color="pink",ax=ax3)
sns.distplot(a=df["Age"].loc[(df["Gender"]==1)],bins=20,color="skyblue",ax=ax3)

sns.distplot(a=df["Annual Income (k$)"].loc[(df["Gender"]==0)],bins=20,color="pink",ax=ax4)
sns.distplot(a=df["Annual Income (k$)"].loc[(df["Gender"]==1)],bins=20,color="skyblue",ax=ax4)

plt.tight_layout(pad=1,h_pad=1,w_pad=1)
plt.suptitle("Gender Distribution",y=1.01,size=16)

* Although they share the same median spending score, females spend slightly more than males.
* There are more females in the customer pool than males with most females being aged between 30 and 40.
* Not much disparity in annual income between the sexes.

In [ ]:
fig=plt.figure(figsize=(12,7))

ax1=plt.subplot2grid((2,3),(0,0),colspan=2,rowspan=2)
ax2=plt.subplot2grid((2,3),(0,2))
ax3=plt.subplot2grid((2,3),(1,2))

sns.scatterplot(x=df["Age"],y=df["Spending Score (1-100)"],hue=df["Gender"],palette={0:"pink",1:"skyblue"},ax=ax1)

sns.distplot(a=df["Age"],bins=20,color="lightsalmon",ax=ax2)

sns.scatterplot(x=df["Age"],y=df["Annual Income (k$)"],color="lightgreen",ax=ax3)

plt.suptitle("Age Distribution",y=1.01,size=16)
plt.tight_layout(pad=1,h_pad=1,w_pad=1)

* People younger than 40 tend to spend more than those older than 40; and more customers younger than 40.
* No relationship between age and annual income, so those younger than 40 are not necessarily earning more but just have more disposable income.

In [ ]:
fig=plt.figure(figsize=(12,5))

ax1=fig.add_subplot(1,2,1)
ax2=fig.add_subplot(1,2,2)

sns.scatterplot(x=df["Annual Income (k$)"],y=df["Spending Score (1-100)"],color="mediumpurple",ax=ax1)

sns.distplot(a=df["Annual Income (k$)"],bins=30,color="lightgreen",ax=ax2)

plt.suptitle("Annual Income Distribution",y=0.93,size=16)

* Those with an annual income of less than 40,000 either spend less or more, but not average.
* Those with an annual income of between 40,000 and 70,000 spend the average amount.
* And those with an annual income of more than 70,000 again either spend less or more, but not average.

Before going further with clustering, it is important to ensure there are no biases and so the data must be scaled:

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
scaled_df=pd.DataFrame(scaler.fit_transform(df),columns=df.columns)
scaled_df.head()

In [ ]:
from sklearn.cluster import KMeans

KMeans requires us to indiate the number of clusters the algorithm should identify. So let's try looking for the elbow point in their inertia (remember the inertia is the sum of squared distances within the cluster - thus the smaller the inertia the better):

In [ ]:
kinertia=[]

for i in range(1,11):
    kmodel=KMeans(n_clusters=i)
    kmodel.fit(scaled_df.drop(["Gender"],axis=1))
    kinertia.append(kmodel.inertia_)

kinertia_df=pd.DataFrame({"No of Cluster":range(1,11),"Inertia":kinertia})

sns.lineplot(x=kinertia_df["No of Cluster"],y=kinertia_df["Inertia"],color="powderblue",linewidth=2,marker="s",markersize=8,markeredgecolor="whitesmoke",markeredgewidth=1)

The optimal number of clusters using their inertia is not very clear so let's compare this to their silhouette scores (remember the silhouette coefficient measures how close each point in a cluster is to the points in its neighbouring clusters - thus the closer the value to +1 the better):

In [ ]:
from sklearn.metrics import silhouette_score

In [ ]:
ksilhouette=[]

for j in range(2,11):
    kmodel=KMeans(n_clusters=j)
    kpredict=kmodel.fit_predict(scaled_df.drop(["Gender"],axis=1))
    kscores=silhouette_score(scaled_df.drop(["Gender"],axis=1),kpredict)
    ksilhouette.append(kscores)

silhouette_df=pd.DataFrame({"no_k":list(range(2,11)),"silhoutte":ksilhouette})

sns.barplot(x=silhouette_df["no_k"],y=silhouette_df["silhoutte"],color="mistyrose")
plt.xlabel("Number of Clusters")
plt.ylabel("Silhouette Score")

Optimal number of clusters should be 6!

In [ ]:
model=KMeans(n_clusters=6,random_state=7)
model.fit(scaled_df)
df["Cluster"]=model.labels_

Now we can examine our 6 mall customer clusters:

In [ ]:
cluster_df=df.groupby(["Cluster"]).mean().T
cluster_df["Average"]=df.apply(lambda x:x.mean(),axis=0)
cluster_df

Cluster 0: relatively young cluster with a high annual income and correspondingly high spending score (the highest spending score).

Cluster 1: second youngest cluster with a slightly lower than average annual income who spends more than their annual income, more than the average spending score.

Cluster 2: the youngest cluster with the lowest annual income spending who again spends more than their annual income, more than the average spending score.

Cluster 3: average aged cluster with the highest annual income but lowest spending score.

Cluster 4: second oldest cluster with a slightly lower than average annual income with a medium spending score.

Cluster 5: the oldest cluster with the second lowest annual income with a medium spending score.

In [ ]:
cluster_df_graphing=cluster_df.drop(["Gender"])

fig,ax=plt.subplots(3,2,figsize=(12,6))

for i,ax in enumerate(ax.flat):
    sns.scatterplot(x=cluster_df_graphing["Average"],y=cluster_df_graphing.index,marker="X",color="k",ax=ax)
    sns.barplot(x=cluster_df_graphing[i],y=cluster_df_graphing.index,palette={"Age":"lightsalmon","Annual Income (k$)":"lightgreen","Spending Score (1-100)":"#AE9CCD"},ax=ax)
    ax.set_xlim(1,90)
    plt.setp(ax.collections,zorder=100)
    ax.set_xlabel("Cluster {}".format(i))
    
plt.tight_layout(pad=1,h_pad=1,w_pad=1)    

In [ ]:
from sklearn.decomposition import PCA
pca=PCA(n_components=2)

pca_df=pd.DataFrame(pca.fit_transform(scaled_df),columns=["PC1","PC2"])
pca_df["Cluster"]=model.labels_

plt.figure(figsize=(12,8))
sns.scatterplot(x=pca_df["PC1"],y=pca_df["PC2"],hue=pca_df["Cluster"],palette="Pastel1")